In [1]:
import attr
import os
import ujson
import bz2
import random

import pandas as pd

from glob import glob
from tqdm import tqdm
from multiprocessing import Pool
from itertools import islice
from functools import partial
from boltons.iterutils import pairwise
from collections import Counter
from scipy import stats
from sklearn.model_selection import train_test_split

from IPython.display import display, Markdown

In [2]:
def zip_offset(seq):
    """Yield (item, 0-1 offset).
    """
    size = len(seq)
    for i, item in enumerate(seq):
        offset = i / (size - 1) if (size - 1) else 0
        yield item, offset

In [3]:
def map_segment(func, path):
    """Parse JSON segment, apply worker function.
    """
    results = []
    
    with bz2.open(path) as fh:
        for line in fh:
            results.append(func(ujson.loads(line)))
            
    return results

In [4]:
@attr.s
class Corpus:
    
    root = attr.ib()
    
    def paths(self):
        return glob(os.path.join(self.root, '*.bz2'))
                        
    def map_novels(self, func, shuffle=True):
        """Apply a worker to segment files in parallel.
        """
        paths = self.paths()
        
        if shuffle:
            random.shuffle(paths)
        
        with Pool() as p:
            
            worker = partial(map_segment, func)
            
            for results in p.imap_unordered(worker, paths):
                yield from results

In [5]:
corpus = Corpus('../../data/chicago-bins-tokens.json/')

In [6]:
def _match_suffixes(root, group, o1, o2, novel, width=5):
    """Match suffix windows after a query token.
    """
    results = []
    
    for i, (t, offset) in enumerate(zip_offset(novel['tokens'])):
        if o1 < offset < o2 and t.lower() == root:
            
            suffix = novel['tokens'][i+1:i+1+width]
            
            results.append((novel['identifier'], group, root, suffix))
    
    return results

In [7]:
def match_suffixes(root, group, o1, o2, width=5, skim=None):
    
    worker = partial(_match_suffixes, root, group, o1, o2, width=width)

    results = corpus.map_novels(worker)
    results = tqdm(islice(results, skim))

    return [r for b in results for r in b]

In [8]:
a0 = match_suffixes('a', 0, 0, 0.05, skim=None)

6638it [03:41, 29.90it/s]


In [9]:
a1 = match_suffixes('a', 1, 0.95, 1, skim=None)

6638it [03:09, 35.08it/s]


In [10]:
the0 = match_suffixes('the', 0, 0, 0.05, skim=None)

6638it [03:25, 32.34it/s]


In [11]:
the1 = match_suffixes('the', 1, 0.95, 1, skim=None)

6638it [03:16, 33.86it/s]


In [12]:
rows = a0 + a1 + the0 + the1
df = pd.DataFrame(rows, columns=('chicago_id', 'group', 'root', 'suffix'))

In [13]:
df

,chicago_id,group,root,suffix
0,23888,0,a,"[small, ,, slight, woman, ,]"
1,23888,0,a,"[gentle, yet, intense, liveliness, about]"
2,23888,0,a,"[voice, that, sounded, ,, as]"
3,23888,0,a,"[sphere, of, red, -, eyed]"
4,23888,0,a,"[minute, ,, "", Ysaye, Barnett]"
5,23888,0,a,"[push, ., The, gravity, ,]"
6,23888,0,a,"[death, grip, on, the, rail]"
7,23888,0,a,"[planet, she, could, settle, on]"
8,23888,0,a,"[tiny, cubicle, like, her, shipboard]"
9,23888,0,a,"[cultural, anthropologist, ., Ysaye, ,]"


In [14]:
df0 = df[df.group==0]

train0, test0 = train_test_split(df0)

train0.to_json('train0.json', orient='records', lines=True)
test0.to_json('test0.json', orient='records', lines=True)

In [15]:
df1 = df[df.group==1]

train1, test1 = train_test_split(df1)

train1.to_json('train1.json', orient='records', lines=True)
test1.to_json('test1.json', orient='records', lines=True)

In [16]:
len(df0)

2868549

In [17]:
len(df1)

2610694